In [4]:
import pandas as pd
from sqlalchemy import create_engine
from config import dbuser, dbpasswd, dburi, dbport
import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session

# Cleaning the Data

Cleaning the Data

In [5]:
# The path to our CSV file
file = "Sobrepeso_Obesidad_Adulta.csv"

# Read our Kickstarter data into pandas
database = pd.read_csv(file, encoding="ISO-8859-1")
database.head()

,folio_c,intp,entidad,locali,sexo,edad,code_upm,est_dis,est_urb,est_marg,...,escolari,munici,imc_ob,a1001b,a1002b,a1002bsp,a1005b,a1006b,indicef,region_2
0,1060001,1,1.0,1.0,1.0,42.0,M0101,5.0,3.0,1.0,...,2.0,1.0,3.0,2.0,NaN,NaN,NaN,NaN,3.310654,2.0
1,1060003,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.310654,NaN
2,1060005,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.397098,NaN
3,1060006,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.397098,NaN
4,1060007,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.644975,NaN


In [6]:
database.columns

Index(['folio_c', 'intp', 'entidad', 'locali', 'sexo', 'edad', 'code_upm',
       'est_dis', 'est_urb', 'est_marg', 'pondef', 'est_var', 'peso', 'talla',
       'imc', 'imc_cat', 'edad_cat', 'ccintura', 'ob_abd', 'nsef', 'nse5f',
       'nse10f', 'area', 's_salud', 'region', 'afilia_1', 'afilia2',
       'escolari', 'munici', 'imc_ob', 'a1001b', 'a1002b', 'a1002bsp',
       'a1005b', 'a1006b', 'indicef', 'region_2'],
      dtype='object')

In [7]:
sobrepeso = database.loc[:,["folio_c", "intp", "entidad", "sexo", "edad", "peso", "talla", "imc", "imc_cat", 
                            "edad_cat", "ob_abd"]]
sobrepeso.head()

,folio_c,intp,entidad,sexo,edad,peso,talla,imc,imc_cat,edad_cat,ob_abd
0,1060001,1,1.0,1.0,42.0,75.0,166.550003,27.03784,3.0,3.0,1.0
1,1060003,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1060005,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1060006,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1060007,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
sobrepeso.count()

folio_c     50756
intp        50756
entidad     41572
sexo        41572
edad        41572
peso        38267
talla       38218
imc         37580
imc_cat     37580
edad_cat    41572
ob_abd      36820
dtype: int64

In [12]:
sobrepeso = sobrepeso.dropna(how='any')
sobrepeso.head()

,folio_c,intp,entidad,sexo,edad,peso,talla,imc,imc_cat,edad_cat,ob_abd
0,1060001,1,1.0,1.0,42.0,75.000000,166.550003,27.037840,3.0,3.0,1.0
6,1060008,1,1.0,2.0,36.0,62.299999,148.250000,28.346445,3.0,2.0,1.0
12,1060015,1,1.0,1.0,35.0,87.400002,168.899994,30.637419,4.0,2.0,1.0
14,1060021,2,1.0,2.0,48.0,54.599998,157.800003,21.926970,2.0,3.0,0.0
19,1060031,4,1.0,2.0,22.0,52.650002,163.750000,19.635220,2.0,1.0,0.0


In [13]:
sobrepeso.count()

folio_c     36675
intp        36675
entidad     36675
sexo        36675
edad        36675
peso        36675
talla       36675
imc         36675
imc_cat     36675
edad_cat    36675
ob_abd      36675
dtype: int64

# Uploading into MySQL

In [14]:
Base = declarative_base()

In [16]:
engine = create_engine(f"mysql://{dbuser}:{dbpasswd}@{dburi}:{dbport}/sobrepeso_mexico_db")
Base.metadata.create_all(engine)

C:\Users\maluk\Anaconda3\envs\malucastellanos\lib\site-packages\pymysql\cursors.py:170: Warning: (1366, "Incorrect string value: '\\xE1ndar ...' for column 'VARIABLE_VALUE' at row 484")
  result = self._query(query)


In [17]:
sobrepeso.to_sql(name='sobrepeso', con=engine, if_exists='append', index=True)

In [18]:
engine.table_names()

['sobrepeso']

In [19]:
data=engine.execute('SELECT * FROM sobrepeso limit 10')
for row in data:
    print(row)

(0, 1060001, 1, 1.0, 1.0, 42.0, 75.0, 166.5500031, 27.03783989, 3.0, 3.0, 1.0)
(6, 1060008, 1, 1.0, 2.0, 36.0, 62.29999924, 148.25, 28.34644508, 3.0, 2.0, 1.0)
(12, 1060015, 1, 1.0, 1.0, 35.0, 87.40000153, 168.8999939, 30.63741875, 4.0, 2.0, 1.0)
(14, 1060021, 2, 1.0, 2.0, 48.0, 54.59999847, 157.8000031, 21.92696953, 2.0, 3.0, 0.0)
(19, 1060031, 4, 1.0, 2.0, 22.0, 52.65000153, 163.75, 19.63521957, 2.0, 1.0, 0.0)
(20, 1060032, 2, 1.0, 2.0, 54.0, 57.59999847, 144.0, 27.77777672, 3.0, 4.0, 1.0)
(21, 1060033, 2, 1.0, 2.0, 35.0, 69.25, 156.3000031, 28.34665489, 3.0, 2.0, 1.0)
(22, 1060034, 6, 1.0, 1.0, 20.0, 55.40000153, 176.0500031, 17.87465668, 1.0, 1.0, 0.0)
(25, 1060037, 3, 1.0, 2.0, 24.0, 66.80000305, 153.6999969, 28.27670288, 3.0, 1.0, 1.0)
(27, 1060039, 1, 1.0, 1.0, 60.0, 67.90000153, 171.3999939, 23.11256599, 2.0, 5.0, 0.0)
